In [1]:
# Dataset comprises of 20k newsposts in 20 different newsgroups - what topics are they on?
# So, this will be a 20-class classification problem!
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

import numpy as np
import nltk
import gensim

In [25]:
# The simplest feature extraction is to use bow - i.e. extract counts of unigrams
# Instead of a frequency bow, we can also use n-gram bow model
# Try to extract trigram bows
def bow_extractor(corpus, ngram_range=(1,1)):
    
    vectorizer = CountVectorizer(min_df=1, ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features


# Weighting each term by how important it is in the document collection
def tfidf_transformer(bow_matrix):
    
    transformer = TfidfTransformer(norm='l2',
                                   smooth_idf=True,
                                   use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix


# TfidfVectorizer is the powerful equivalent of CountVectorizer for TF-IDF weights
def tfidf_extractor(corpus, ngram_range=(1,1)):
    
    vectorizer = TfidfVectorizer(min_df=1, 
                                 norm='l2',
                                 smooth_idf=True,
                                 use_idf=True,
                                 ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features


# Two weighting schemes for combining word vectors in documents
# 1. averaging the word vectors
# 2. tf-idf weighting of word vectors

# Averaging word vectors of a document
def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)


# Using tfidf weighted average of word vectors in a document  
def tfidf_wtd_avg_word_vectors(words, tfidf_vector, tfidf_vocabulary, model, num_features):
    
    word_tfidfs = [tfidf_vector[0, tfidf_vocabulary.get(word)] 
                   if tfidf_vocabulary.get(word) 
                   else 0 for word in words]    
    word_tfidf_map = {word:tfidf_val for word, tfidf_val in zip(words, word_tfidfs)}
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    vocabulary = set(model.wv.index_to_key)
    wts = 0.
    for word in words:
        if word in vocabulary: 
            word_vector = model.wv[word]
            weighted_word_vector = word_tfidf_map[word] * word_vector
            wts = wts + word_tfidf_map[word]
            feature_vector = np.add(feature_vector, weighted_word_vector)
    if wts:
        feature_vector = np.divide(feature_vector, wts)
        
    return feature_vector

def tfidf_weighted_averaged_word_vectorizer(corpus, tfidf_vectors, 
                                   tfidf_vocabulary, model, num_features):
                                       
    docs_tfidfs = [(doc, doc_tfidf) 
                   for doc, doc_tfidf 
                   in zip(corpus, tfidf_vectors)]
    features = [tfidf_wtd_avg_word_vectors(tokenized_sentence, tfidf, tfidf_vocabulary,
                                   model, num_features)
                    for tokenized_sentence, tfidf in docs_tfidfs]
    return np.array(features)

In [3]:
# remove headers of email since they won't help us in 
def get_data():
    data = fetch_20newsgroups(subset='all',
                              shuffle=True,
                              remove=('headers', 'footers', 'quotes'))
    return data

In [4]:
# Divide the data into training and testing sets
def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels, test_size=0.33, random_state=42)
    return train_X, test_X, train_Y, test_Y

In [5]:
# remove empty documents since they would just add noise
def remove_empty_docs(corpus, labels):
    filtered_corpus = []
    filtered_labels = []
    for doc, label in zip(corpus, labels):
        if doc.strip():
            filtered_corpus.append(doc)
            filtered_labels.append(label)

    return filtered_corpus, filtered_labels

In [6]:
dataset = get_data()

In [7]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [8]:
corpus, labels = dataset.data, dataset.target
corpus, labels = remove_empty_docs(corpus, labels)

In [9]:
print('Sample document:', corpus[10])

Sample document: the blood of the lamb.

This will be a hard task, because most cultures used most animals
for blood sacrifices. It has to be something related to our current
post-modernism state. Hmm, what about used computers?

Cheers,
Kent


In [10]:
print('Class label:',labels[10])

Class label: 19


In [11]:
print('Actual class label:', dataset.target_names[labels[10]])

Actual class label: talk.religion.misc


In [12]:
train_corpus, test_corpus, train_labels, test_labels = prepare_datasets(corpus, labels, test_data_proportion=0.3)

In [13]:
# First normalize both the training and test data using our previous funcionts 
from normalization import normalize_corpus

norm_train_corpus = normalize_corpus(train_corpus)
norm_test_corpus = normalize_corpus(test_corpus)  

''.strip()

''

In [14]:
# Extract features using the extractors we defined

In [15]:
# Bag of words (BoW) features
bow_vectorizer, bow_train_features = bow_extractor(norm_train_corpus)  
bow_test_features = bow_vectorizer.transform(norm_test_corpus)

In [16]:
# TFIDF features
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)
tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)

In [17]:
# tokenize documents
tokenized_train = [nltk.word_tokenize(text)
                   for text in norm_train_corpus]
tokenized_test = [nltk.word_tokenize(text)
                   for text in norm_test_corpus] 

In [18]:
# build word2vec model 
model = gensim.models.Word2Vec(tokenized_train, vector_size=500, window=100, min_count=30, sample=1e-3)

In [19]:
# averaged word vector features
avg_wv_train_features = averaged_word_vectorizer(corpus=tokenized_train, model=model, num_features=500)
avg_wv_test_features = averaged_word_vectorizer(corpus=tokenized_test, model=model, num_features=500)

In [47]:
avg_wv_train_features[0][0]

-0.22770961540702142

In [26]:
# tfidf weighted averaged word vector features
vocab = tfidf_vectorizer.vocabulary_
tfidf_wv_train_features = tfidf_weighted_averaged_word_vectorizer(corpus=tokenized_train, 
                                                                  tfidf_vectors=tfidf_train_features,
                                                                  tfidf_vocabulary=vocab, model=model, num_features=500)
tfidf_wv_test_features = tfidf_weighted_averaged_word_vectorizer(corpus=tokenized_test, tfidf_vectors=tfidf_test_features, 
                                                                 tfidf_vocabulary=vocab, model=model, num_features=500)

In [27]:
# Use sklearn's metrics function for evaluation of classifiers

In [28]:
# Define function to calculate the 4 common mertics

def get_metrics(true_labels, predicted_labels):
    print('Accuracy:', np.round(metrics.accuracy_score(true_labels, predicted_labels), 2))
    print('Precision:', np.round(metrics.precision_score(true_labels, predicted_labels, average='weighted'), 2))
    print('Recall:', np.round(metrics.recall_score(true_labels, predicted_labels, average='weighted'), 2))
    print('F1 Score:', np.round(metrics.f1_score(true_labels, predicted_labels, average='weighted'), 2))

In [29]:
# Master function to call the above defined functions to perform the classification,
# predict the results and evaluate predictions against the test data
def train_predict_evaluate_model(classifier, train_features, train_labels, test_features, test_labels): 
    # build model    
    classifier.fit(train_features, train_labels)
    
    # predict using model
    predictions = classifier.predict(test_features)
    
    # evaluate model prediction performance 
    get_metrics(true_labels=test_labels, predicted_labels=predictions)
    return predictions

In [30]:
# Import the two classification algorithms we want to use for the task
# Based on the features we extract, we altogether have 6 combinations of models to train
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

mnb = MultinomialNB() # Multinomial Naive Bayes
svm = SGDClassifier(loss='hinge', max_iter=100) # Support Vector Machine

In [31]:
# Multinomial Naive Bayes with bag of words features
mnb_bow_predictions = train_predict_evaluate_model(classifier=mnb, 
                                                   train_features=bow_train_features, 
                                                   train_labels=train_labels, 
                                                   test_features=bow_test_features, 
                                                   test_labels=test_labels)

Accuracy: 0.67
Precision: 0.73
Recall: 0.67
F1 Score: 0.65


In [32]:
# Support Vector Machine with bag of words features
svm_bow_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=bow_train_features,
                                           train_labels=train_labels,
                                           test_features=bow_test_features,
                                           test_labels=test_labels)

Accuracy: 0.64
Precision: 0.68
Recall: 0.64
F1 Score: 0.65


In [33]:
# Multinomial Naive Bayes with tfidf features
mnb_tfidf_predictions = train_predict_evaluate_model(classifier=mnb,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels)

Accuracy: 0.72
Precision: 0.78
Recall: 0.72
F1 Score: 0.71


In [34]:
# Support Vector Machine with tfidf features
svm_tfidf_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=tfidf_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features,
                                           test_labels=test_labels)

Accuracy: 0.77
Precision: 0.77
Recall: 0.77
F1 Score: 0.76


In [35]:
# Support Vector Machine with averaged word vector features
svm_avgwv_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=avg_wv_train_features,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features,
                                           test_labels=test_labels)

Accuracy: 0.53
Precision: 0.57
Recall: 0.53
F1 Score: 0.51


In [36]:
# Support Vector Machine with tfidf weighted averaged word vector features
svm_tfidfwv_predictions = train_predict_evaluate_model(classifier=svm,
                                           train_features=tfidf_wv_train_features,
                                           train_labels=train_labels,
                                           test_features=tfidf_wv_test_features,
                                           test_labels=test_labels)

Accuracy: 0.53
Precision: 0.55
Recall: 0.53
F1 Score: 0.52


In [37]:
# Confusion matrix for the tfidf-based SVM model (best in this case?)
import pandas as pd
cm = metrics.confusion_matrix(test_labels, svm_tfidf_predictions)
pd.DataFrame(cm, index=range(0,20), columns=range(0,20))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,157,2,0,1,1,2,2,3,3,0,5,3,3,6,5,31,4,7,8,20
1,1,225,9,7,6,16,6,2,2,1,0,3,5,3,4,2,2,0,3,0
2,1,19,219,20,7,21,9,1,0,0,0,3,6,2,1,2,0,1,2,0
3,1,12,24,222,10,4,9,2,1,1,1,2,6,3,1,0,1,0,0,0
4,1,5,8,14,226,4,4,2,3,1,0,3,11,3,4,1,1,0,1,0
5,0,21,18,2,1,270,1,0,1,0,0,0,4,4,1,1,0,1,0,0
6,0,2,7,10,12,1,270,10,3,2,1,1,10,1,4,0,2,2,1,0
7,3,4,2,0,2,4,4,246,21,1,2,1,9,4,2,2,2,3,4,0
8,4,0,0,4,3,2,4,25,253,3,2,3,1,5,2,3,1,2,5,0
9,2,1,1,0,2,3,5,5,5,274,13,1,2,1,2,3,2,1,2,0


In [38]:
class_names = dataset.target_names
print(class_names[0], '->', class_names[15])
print(class_names[18], '->', class_names[16]) 
print(class_names[19], '->', class_names[15])

alt.atheism -> soc.religion.christian
talk.politics.misc -> talk.politics.guns
talk.religion.misc -> soc.religion.christian


In [39]:
# Checking the misclassified documents for error analysis
import re

num = 0
for document, label, predicted_label in zip(test_corpus, test_labels, svm_tfidf_predictions):
    if label == 0 and predicted_label == 15:
        print('Actual Label:', class_names[label])
        print('Predicted Label:', class_names[predicted_label])
        print('Document:-')
        print(re.sub('\n', ' ', document))
        print()
        
        num += 1
        if num == 4:
            break

Actual Label: alt.atheism
Predicted Label: soc.religion.christian
Document:-
I would like a list of Bible contadictions from those of you who dispite being free from Christianity are well versed in the Bible. 

Actual Label: alt.atheism
Predicted Label: soc.religion.christian
Document:-
  They spent quite a bit of time on the wording of the Constitution.  They picked words whose meanings implied the intent.  We have already looked in the dictionary to define the word.  Isn't this sufficient?   But we were discussing it in relation to the death penalty.  And, the Constitution need not define each of the words within.  Anyone who doesn't know what cruel is can look in the dictionary (and we did).

Actual Label: alt.atheism
Predicted Label: soc.religion.christian
Document:-
Our Lord and Savior David Keresh has risen!   	He has been seen alive!   	Spread the word!     --------------------------------------------------------------------------------

Actual Label: alt.atheism
Predicted Label

In [40]:
num = 0
for document, label, predicted_label in zip(test_corpus, test_labels, svm_tfidf_predictions):
    if label == 18 and predicted_label == 16:
        print('Actual Label:', class_names[label])
        print('Predicted Label:', class_names[predicted_label])
        print('Document:-')
        print(re.sub('\n', ' ', document))
        print()
        num += 1
        if num == 4:
            break

Actual Label: talk.politics.misc
Predicted Label: talk.politics.guns
Document:-
After the initial gun battle was over, they had 50 days to come out peacefully. They had their high priced lawyer, and judging by the posts here they had some public support. Can anyone come up with a rational explanation why the didn't come out (even after they negotiated coming out after the radio sermon) that doesn't include the Davidians wanting to commit suicide/murder/general mayhem?

Actual Label: talk.politics.misc
Predicted Label: talk.politics.guns
Document:-
Yesterday, the FBI was saying that at least three of the bodies had gunshot wounds, indicating that they were shot trying to escape the fire.  Today's paper quotes the medical examiner as saying that there is no evidence of gunshot wounds in any of the recovered bodies.  At the beginning of this siege, it was reported that while Koresh had a class III (machine gun) license, today's paper quotes the government as saying, no, they didn't have a